---
title: "Karte"
jupyter: python3
#code-fold: true
execute:
  echo: false
  output: asis
---

In [7]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pprint 
from pathlib import Path
from collections import defaultdict
import csv
from geopy.geocoders import Nominatim
import time
from functools import lru_cache
from pathlib import Path
import folium
from statistics import mean
from IPython.display import display

def query_WB(endpoint, query):

    # Initialize the wrapper
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Fetch and parse results
    results = sparql.query().convert()

    return results 

def get_data(results):

    return [
        {
            "url":           result["building"]["value"],
            "label":         result["buildingLabel"]["value"],
            "description":   result["buildingDescription"]["value"],
            "photo":         result.get("img",{}).get("value",""),
            "documented_at": result.get("docu",{}).get("value",""),
            "address":       result.get("address",{}).get("value",""),
        }
        for result in results["results"]["bindings"]
    ]


def trans_addresses(data):

    cache_path = Path("buildings_map.csv")
    # load existing cache from CSV
    cache = {}
    if cache_path.exists():
        with open(cache_path, newline="", encoding='ISO 8859-1') as f:
            reader = csv.reader(f)
            for addr, lat, lon in reader:
                cache[addr] = (float(lat), float(lon))
                
    @lru_cache(maxsize=None)
    def geocode(addr):
        if addr in cache:
            return cache[addr]
        loc = Nominatim(user_agent="my_app").geocode(addr)
        time.sleep(1)
        if loc:
            cache[addr] = (loc.latitude, loc.longitude)
            # immediately append new rows to CSV
            with open(cache_path, "a", newline="", encoding="ISO 8859-1") as f:
                writer = csv.writer(f)
                writer.writerow([addr, loc.latitude, loc.longitude])
            return cache[addr]
        return (None, None)

    for item in data:
        lat, lon = geocode(item["address"])
        item["lat"], item["lon"] = lat, lon
    
    return data
    
def is_supported_image(path):
    ext = Path(path).suffix.lower()
    return ext in [".jpg", ".jpeg", ".png"]

def generate_output(data):

    center_lat = mean(item["lat"] for item in data)
    center_lon = mean(item["lon"] for item in data)

    # 3) Create the map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

    # 4) Add markers
    for item in data:
        folium.Marker(
            [item["lat"], item["lon"]],
            popup=item["address"],
            tooltip=item["label"]
        ).add_to(m)

    # 5) Display
    m.save("buildings_map.html")
    display(m)



In [8]:
def make_book():

    endpoint_url = "https://query.kewl.org/sparql"

    query = """
PREFIX wd: <https://wikibase.kewl.org/entity/>
PREFIX wdt: <https://wikibase.kewl.org/prop/direct/>
PREFIX p: <https://wikibase.kewl.org/prop/>
PREFIX ps: <https://wikibase.kewl.org/prop/statement/>
PREFIX pq: <https://wikibase.kewl.org/prop/qualifier/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT DISTINCT ?building ?buildingLabel ?buildingDescription ?address (SAMPLE(?docus) AS ?docu) (SAMPLE(?imgs) AS ?img) WHERE {
  # get all entries that have an address
  ?building wdt:P5 ?address;
            wdt:P7 ?docus;
            wdt:P6 ?imgs .
  
  # minus those with a "part of" attribute
  MINUS { ?building wdt:P3 ?parent . }
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de,en". }
} GROUP BY 
  ?building
  ?buildingLabel
  ?buildingDescription
  ?address
    """

    results = query_WB(endpoint_url,query)
    data = get_data(results)
    newdata = trans_addresses(data)
    generate_output(newdata)

make_book()


